In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

In [2]:
def load_and_preprocess():
    (X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
    X_test = X_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0
    Y_train = tf.one_hot(Y_train, depth=10)
    Y_test = tf.one_hot(Y_test, depth=10)
    X_train = tf.concat([X_train, X_test], axis=0)
    Y_train = tf.concat([Y_train, Y_test], axis=0)
    return X_train, Y_train

In [3]:
X_train, Y_train = load_and_preprocess()

In [4]:
def create_and_compile_model():
    
    inputs = tf.keras.layers.Input(shape=(28,28,1))
    x = tf.keras.layers.Conv2D(64, kernel_size=(3,3))(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(256)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(10)(x)
    x = tf.keras.activations.softmax(x)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    
    model.compile(
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics = [tf.keras.metrics.CategoricalAccuracy()])
    
    return model

In [5]:
model = create_and_compile_model()

In [11]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                              min_delta=0.0001, patience=5, restore_best_weights=True)]

model.fit(X_train, Y_train, epochs=50, batch_size=64, callbacks=callbacks)

Epoch 1/50
1094/1094 [==============================] - 7s 6ms/step - loss: 0.0578 - categorical_accuracy: 0.9821
Epoch 2/50
1094/1094 [==============================] - 11s 10ms/step - loss: 0.0582 - categorical_accuracy: 0.9813
Epoch 3/50
1094/1094 [==============================] - 12s 11ms/step - loss: 0.0586 - categorical_accuracy: 0.9813
Epoch 4/50
1094/1094 [==============================] - 10s 9ms/step - loss: 0.0598 - categorical_accuracy: 0.9813
Epoch 5/50
1094/1094 [==============================] - 15s 14ms/step - loss: 0.0603 - categorical_accuracy: 0.9814
Epoch 6/50
1094/1094 [==============================] - 15s 14ms/step - loss: 0.0608 - categorical_accuracy: 0.9812


In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 256)               200960    
                                                                 
 batch_normalization_2 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 tf.nn.relu_2 (TFOpLambda)   (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

In [13]:
model.save("DigiPredictor_final.h5")